In [63]:
import tensorflow as tf
import json
import numpy as np
import cv2

In [4]:
labels = tf.data.Dataset.list_files(r'D:\Competitions\ComputerVision\OCR\Signalboard\dataset\train\label\official\*.json')

In [64]:
decode_json(r'D:\Competitions\ComputerVision\OCR\Signalboard\dataset\train\label\official\img_1.json')

array([[ 805,  613,  255,   53],
       [ 692,  557,  106,  111],
       [ 701, 1016,    8,    9],
       [ 694, 1008,   18,   12],
       [ 692,  956,   17,   56],
       [ 806,  620,   46,   47],
       [ 957,  615,   48,   50],
       [ 908,  619,   43,   40],
       [1012,  616,   47,   48],
       [ 852,  618,   44,   47],
       [ 693,  992,   17,   20],
       [ 694,  974,   15,   20],
       [ 694,  957,   15,   18]])

In [61]:
def decode_json(filepath):
    byte = tf.io.read_file(filepath)
    label = json.loads(byte.numpy())
    
    H = int(label['imageHeight'])
    W = int(label['imageWidth'])
    
    polygon = np.array([p['points'] for p in label['shapes']])
    #因為標記座標可能超出圖片範圍，所以要修正
    polygon[:,:,0] = np.clip(polygon[:,:,0],0,W)
    polygon[:,:,1] = np.clip(polygon[:,:,1],0,H)
    
    xywh = np.array([cv2.boundingRect(p) for p in polygon])
    
    polygon = polygon/np.array([W,H])
    xywh = xywh/np.array([W,H,W,H])
    center = np.stack([xywh[:,0]+xywh[:,2]/2, xywh[:,1]+xywh[:,3]/2],axis=-1)
    
    ul = np.stack(xywh[:,0],xywh[:,1],axis=-1)
    ur = np.stack(xywh[:,0]+xywh[:,2],xywh[:,1],axis=-1)
    dr = np.stack(xywh[:,0]+xywh[:,2],xywh[:,1]+xywh[:,3],axis=-1)
    dl = np.stack(xywh[:,0],xywh[:,1]+xywh[:,3],axis=-1)
    rect = np.stack([ul,ur,dr,dl],axis=1)
    
    offset = polygon - rect
    
    group = np.array([p['group_id'] for p in label['shapes']])
    
#     bboxes = []
#     for shape in official_label['shapes']:
#         points = np.array(shape['points'])
#         
#         points[:,0] = np.clip(points[:,0],0,W)
#         points[:,1] = np.clip(points[:,1],0,H)        
#         x,y,w,h = cv2.boundingRect(points)
        
#         #四邊形座標依照圖片尺寸正規化
#         points = points/np.array([W,H])
#         #矩形bbox的xywh依照圖片尺寸正規化，並且將定位座標從左上角改為bbox中心
#         x,w = x/W, w/W
#         y,h = y/H, h/H
#         x = x+w/2
#         y = y+h/2
        
#         bbox = {}
#         bbox['text'] = shape['label']
#         bbox['group'] = shape['group_id']
#         bbox['points'] = points.tolist()
#         bbox['center'] = points.mean(axis=0).tolist()
#         bbox['xywh'] = [x,y,w,h]
#         bboxes.append(bbox)
        
#     bboxes = sorted(bboxes,key = lambda x: x['center'])
    
#     my_label = {}
#     my_label['ImageName'] = official_label['imagePath']
#     my_label['ImageHeight'] = H
#     my_label['ImageWidth'] = W
#     my_label['bboxes'] = bboxes
#     my_label['bboxes_count'] = len(bboxes)
    
    return xywh

def parse_json(filepath):
    
    [js,] = tf.py_function(decode_json, [byte], [tf.string])
    return js


In [55]:
x = labels.map(parse_json,num_parallel_calls=tf.data.AUTOTUNE)

In [56]:
s = list(x.take(5).as_numpy_iterator())[0]

In [58]:
a = tf.zeros((2,2))

In [59]:
a[0] =1

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [57]:
s

b'img_10992.jpg'

In [12]:
json.loads(s)

{'version': '4.5.7',
 'flags': {},
 'shapes': [{'label': '站',
   'points': [[57, 162], [63, 173], [60, 191], [54, 181]],
   'group_id': 1,
   'shape_type': 'polygon',
   'flags': {}},
  {'label': '電',
   'points': [[50, 150], [56, 162], [53, 180], [47, 170]],
   'group_id': 1,
   'shape_type': 'polygon',
   'flags': {}},
  {'label': '',
   'points': [[23, 145], [51, 194], [49, 202], [20, 158]],
   'group_id': 5,
   'shape_type': 'polygon',
   'flags': {}},
  {'label': '電動機車充電站',
   'points': [[14, 88], [64, 174], [60, 191], [10, 108]],
   'group_id': 0,
   'shape_type': 'polygon',
   'flags': {}},
  {'label': 'KYMCO',
   'points': [[17, 65], [54, 127], [49, 142], [14, 83]],
   'group_id': 2,
   'shape_type': 'polygon',
   'flags': {}},
  {'label': '大',
   'points': [[917, 557], [1101, 559], [1110, 736], [913, 738]],
   'group_id': 1,
   'shape_type': 'polygon',
   'flags': {}},
  {'label': '王',
   'points': [[1133, 558], [1314, 557], [1328, 733], [1130, 735]],
   'group_id': 1,
   'sha